In [ ]:
 !pip install transformers[torch]
!pip install peft
!pip install bitsandbytes
!pip install sacrebleu
!pip install matplotlib
!pip install nltk
!pip install accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 6.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from transformers import MT5ForConditionalGeneration, AutoTokenizer
import pandas as pd
import torch
import numpy as np
from accelerate import Accelerator
import os,shutil

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
torch.cuda.empty_cache()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_name ="./drive/MyDrive/mt5-qlora-200k"

In [ ]:
# Load the Trained Model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)
accelerator = Accelerator()
model = accelerator.prepare(model)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note th

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
model.eval()

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): lora.Linear(
                (base_layer): Linear(in_features=512, out_features=384, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=512, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=384, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k): Linear(in_features=512, out_features=384, bias=False)
         

In [ ]:
def encode_str(text, tokenizer,):
    input_ids = tokenizer.encode(
      text=text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,)
    return input_ids[0]

def random_model_test(model,tokenizer, df_validation,n=10,verbose=True):
    random_test= df_validation.sample(n=n)
    for i in range(len(random_test)):
        en_test_data = random_test.iloc[i].en
        fr_test_data = random_test.iloc[i].fr
        en = encode_str(en_test_data,tokenizer).unsqueeze(0).cuda()
        res  = model.generate(en)
        res_decoded= tokenizer.decode(res[0],skip_special_tokens=True)
        if verbose:
            print('English Sentence:')
            print(en_test_data)
            print('French Sentence:')
            print(fr_test_data)
            print('Model Output:')
            print(res_decoded)
            print('--------\n')

In [ ]:
df= pd.read_csv('test.csv',nrows=10)

In [ ]:
random_model_test(model,tokenizer, df,n=10,verbose=True)

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


English Sentence:
Some donors are planning to go beyond their initial pledges (US, Algeria).
French Sentence:
Certains donateurs prévoient d'aller au-delà de leurs engagements initiaux (États-Unis, Algérie).
Model Output:
<extra_id_0>s ont été planning à l’avance (US, Algeria).
--------

English Sentence:
Other remedial work suggested in the Draft Report be undertaken.  17.
French Sentence:
Que d'autres travaux de restauration proposés dans le rapport provisoire soient entrepris.  17.
Model Output:
<extra_id_0> à l’étude de la Draft Report a été arrêt
--------

English Sentence:
As a result, section 7 of the Charter is inapplicable in the present case.
French Sentence:
Par conséquent, l'article 7 de la Charte est inapplicable dans la présente cause.
Model Output:
<extra_id_0> a été l’article 7 de la Charter est inapplicable en
--------

English Sentence:
Mexico’s ratification affirms the overwhelming support for the International Criminal Court.
French Sentence:
La ratification du Mexi